In [33]:
import sqlite3
import pandas as pd

In [7]:
sqlite3.sqlite_version

'3.32.3'

In [8]:
conn = sqlite3.connect('C:/sqlite/join.db')  

In [9]:
cur = conn.cursor()    

## Table 

In [20]:
cur.execute('select * from topic')
rows = cur.fetchall()
for i in rows:
    print(i)
    
col = list(map(lambda x: x[0], cur.description))
pd.DataFrame(rows, columns=col)

(1, 'HTML', 'HTML is ...', '1')
(2, 'CSS', 'CSS is ...', '2')
(3, 'JavaScript', 'JavaScript is ..', '1')
(4, 'Database', 'Database is ...', None)


,tid,title,description,author_id
0,1,HTML,HTML is ...,1
1,2,CSS,CSS is ...,2
2,3,JavaScript,JavaScript is ..,1
3,4,Database,Database is ...,None


In [21]:
cur.execute('select * from author')
rows = cur.fetchall()
for i in rows:
    print(i)
 
col = list(map(lambda x: x[0], cur.description))
pd.DataFrame(rows, columns=col)

(1, 'egoing', 'seoul', 1)
(2, 'leezche', 'jeju', 2)
(3, 'blackdew', 'namhae', 3)


,aid,name,city,profile_id
0,1,egoing,seoul,1
1,2,leezche,jeju,2
2,3,blackdew,namhae,3


In [22]:
cur.execute('select * from profile')
rows = cur.fetchall()
for i in rows:
    print(i)
    
col = list(map(lambda x: x[0], cur.description))
pd.DataFrame(rows, columns=col)

(1, 'developer', 'developer is ...')
(2, 'designer', 'designer is ..')
(3, 'DBA', 'DBA is ...')


,pid,title,description
0,1,developer,developer is ...
1,2,designer,designer is ..
2,3,DBA,DBA is ...


### left join

In [24]:
cur.execute('select * from topic left join author on topic.author_id = author.aid')
rows = cur.fetchall()
for i in rows:
    print(i)

col = list(map(lambda x : x[0], cur.description)) 
pd.DataFrame(rows, columns = col)

(1, 'HTML', 'HTML is ...', '1', 1, 'egoing', 'seoul', 1)
(2, 'CSS', 'CSS is ...', '2', 2, 'leezche', 'jeju', 2)
(3, 'JavaScript', 'JavaScript is ..', '1', 1, 'egoing', 'seoul', 1)
(4, 'Database', 'Database is ...', None, None, None, None, None)


,tid,title,description,author_id,aid,name,city,profile_id
0,1,HTML,HTML is ...,1,1.0,egoing,seoul,1.0
1,2,CSS,CSS is ...,2,2.0,leezche,jeju,2.0
2,3,JavaScript,JavaScript is ..,1,1.0,egoing,seoul,1.0
3,4,Database,Database is ...,None,NaN,None,None,NaN


In [27]:
cur.execute('select * from topic left join author on topic.author_id = author.aid left join profile on author.aid = profile.pid')
rows = cur.fetchall()

col = list(map(lambda x : x[0], cur.description))
pd.DataFrame(rows, columns= col)

,tid,title,description,author_id,aid,name,city,profile_id,pid,title,description
0,1,HTML,HTML is ...,1,1.0,egoing,seoul,1.0,1.0,developer,developer is ...
1,2,CSS,CSS is ...,2,2.0,leezche,jeju,2.0,2.0,designer,designer is ..
2,3,JavaScript,JavaScript is ..,1,1.0,egoing,seoul,1.0,1.0,developer,developer is ...
3,4,Database,Database is ...,None,NaN,None,None,NaN,NaN,None,None


## exclusive left join 

In [28]:
cur.execute('select * from topic left join author on topic.author_id = author.aid where author.aid is null')
rows = cur.fetchall()
col = list(map(lambda x : x[0], cur.description))
pd.DataFrame(rows, columns= col)

,tid,title,description,author_id,aid,name,city,profile_id
0,4,Database,Database is ...,None,None,None,None,None


## inner join 

In [29]:
cur.execute('select * from topic inner join author on topic.author_id = author.aid')
rows = cur.fetchall()
col = list(map(lambda x : x[0], cur.description))
pd.DataFrame(rows, columns= col)

,tid,title,description,author_id,aid,name,city,profile_id
0,1,HTML,HTML is ...,1,1,egoing,seoul,1
1,2,CSS,CSS is ...,2,2,leezche,jeju,2
2,3,JavaScript,JavaScript is ..,1,1,egoing,seoul,1


### sqlite 3에서는 right join 과 outer join을 지원하지 않는다

## Right join

In [32]:
cur.execute('select tid, title, description, author_id, aid, name, city, profile_id from author left join topic on author.aid = topic.author_id')
rows = cur.fetchall()
col = list(map(lambda x : x[0], cur.description))
pd.DataFrame(rows, columns= col)

,tid,title,description,author_id,aid,name,city,profile_id
0,1.0,HTML,HTML is ...,1,1,egoing,seoul,1
1,3.0,JavaScript,JavaScript is ..,1,1,egoing,seoul,1
2,2.0,CSS,CSS is ...,2,2,leezche,jeju,2
3,NaN,None,None,None,3,blackdew,namhae,3


## Outer join

In [35]:
cur.execute('select * from topic left join author on topic.author_id = author.aid union select tid, title, description, author_id, aid, name, city, profile_id from author left join topic on author.aid = topic.author_id where topic.author_id is null')
rows = cur.fetchall()
col = list(map(lambda x : x[0], cur.description))
pd.DataFrame(rows, columns= col)

,tid,title,description,author_id,aid,name,city,profile_id
0,NaN,None,None,None,3.0,blackdew,namhae,3.0
1,1.0,HTML,HTML is ...,1,1.0,egoing,seoul,1.0
2,2.0,CSS,CSS is ...,2,2.0,leezche,jeju,2.0
3,3.0,JavaScript,JavaScript is ..,1,1.0,egoing,seoul,1.0
4,4.0,Database,Database is ...,None,NaN,None,None,NaN
